## Intermediate Machine Learning

In [23]:
# Preparation :
import pandas as pd
from sklearn.model_selection import train_test_split as split
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_absolute_error 
from sklearn.impute import SimpleImputer as imp

In [21]:
path="./melb_data.csv"
data=pd.read_csv(path)
y=data["Price"]
X = data.select_dtypes(exclude=['object'])
X=X.drop(["Price"],axis=1)
X_train, X_valid, y_train, y_valid = split(X,y,train_size=0.8, test_size=0.2,random_state=1)

In [8]:
# score function
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

## Missing value :
Dealing by using imputer

In [26]:
# Simple imputer
imputer=imp()
def Imputer(X_train,X_valid):
    X_train_i=pd.DataFrame(imputer.fit_transform(X_train))
    X_valid_i=pd.DataFrame(imputer.transform(X_valid))
    # Imputation removed column names; put them back
    X_train_i.columns = X_train.columns
    X_valid_i.columns = X_valid.columns
    return X_train_i,X_valid_i

In [27]:
X_train_i,X_valid_i=Imputer(X_train,X_valid)

In [6]:
print("MAE from Approach  (Imputation):")
print(score_dataset(X_train_i, X_valid_i, y_train, y_valid))

MAE from Approach  (Imputation):
175503.57803141873


In [7]:
# Extension inputer 
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()
cols=X_train_plus.columns
# Make new columns indicating what will be imputed
for col in cols:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()
X_train_i,X_valid_i=Imputer(X_train_plus,X_valid_plus)
print("MAE from Approach  Extension(Imputation):")
print(score_dataset(X_train_i, X_valid_i, y_train, y_valid))

MAE from Approach  Extension(Imputation):
177928.35693421698


## Factors

In [57]:
path="./melb_data.csv"
data=pd.read_csv(path)
y=data["Price"]
X=data.drop(["Price"],axis=1)
X_train_full, X_valid_full, y_train, y_valid = split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# Drop columns with missing values (simplest approach)
cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()] 
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]
# Keep selected columns only
my_cols = low_cardinality_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

/Users/xinyunwu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [58]:
X_train.head()


,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.8272,4217.0


In [51]:
# Label Encoding
from sklearn.preprocessing import LabelEncoder as le 
from sklearn.preprocessing import OneHotEncoder 
Factors
# Disregard : 'Suburb', 'Address' , SellerG, Date,CouncilArea
# LabelEncoder :  
# OneHotEncoder :  'Type', 'Method' ,'Regionname'
X_train.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Method', 'SellerG', 'Date',
       'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize',
       'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude', 'Longtitude',
       'Regionname', 'Propertycount'],
      dtype='object')

In [59]:

# Get list of categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['Type', 'Method', 'Regionname']


In [60]:
from sklearn.preprocessing import LabelEncoder

# Make copy to avoid changing original data 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply label encoder to each column with categorical data
label_encoder = LabelEncoder()
for col in object_cols:
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_valid[col] = label_encoder.transform(X_valid[col])

print("MAE from Approach 2 (Label Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE from Approach 2 (Label Encoding):
175062.2967599411


In [61]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 3 (One-Hot Encoding):
176703.63810751104


## Pipeline
Wonderful LOL : So tired of writing the messy code above

In [2]:

# Separate target from predictors
path="./melb_data.csv"
data=pd.read_csv(path)
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [25]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

In [26]:
from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

ValueError: A given column is not a column of the dataframe

## Cross-validation

In [11]:
from sklearn.model_selection import cross_val_score
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')
print("MAE scores:\n", scores)

MAE scores:
 [207273.036228   195544.72890525 186907.59467108 152084.99219493
 158236.49133232]


In [12]:
print("Average MAE score (across experiments):")
print(scores.mean())

Average MAE score (across experiments):
180009.368666316


## Ensemble models - Gradient 

In [31]:
# using pipline 

from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
model = XGBRegressor(n_estimators=1000,learning_rate=0.05)
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
# model__parameters
my_pipeline.fit(X_train, y_train,model__early_stopping_rounds=10,model__eval_set=[(X_valid, y_valid)],model__verbose=False)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 246847.48380891015


In [ ]:
MAE: 163169.60575639727